In [ ]:
# Reload module to pick up changes
import importlib
import verse_scraper
importlib.reload(verse_scraper)

from verse_scraper import scrape_verse, VerseData
import json

from verse_scraper import BIBLE_BOOKS, scrape_chapter, scrape_book, scrape_bible, scrape_verse_safe
import time

In [ ]:
# Scrape an entire book (be patient - this takes a while!)
# Example: Scrape Philemon (short book with 1 chapter, 25 verses)

def print_progress(verse_data):
    print(f"  Scraped: {verse_data.reference}")

# Uncomment to run:
# philemon_data = scrape_book("philemon", callback=print_progress)
print(json.dumps([v.to_dict() for v in philemon_data],indent=2,ensure_ascii=False))
print(f"\nTotal verses in Philemon: {len(philemon_data)}")

In [ ]:
# Full Bible scraping would take HOURS (~31,000 verses at ~1.3s each = ~11 hours)
# Here's how to do it with saving progress:

import json
import time
from pathlib import Path

def scrape_with_persistence(output_dir: str = "bible_data"):
    """Scrape the Bible with progress saving to disk."""
    Path(output_dir).mkdir(exist_ok=True)
    
    for book in BIBLE_BOOKS:
        output_file = Path(output_dir) / f"{book}.json"
        
        # Skip if already scraped
        if output_file.exists():
            print(f"Skipping {book} (already exists)")
            continue
        
        print(f"\n📖 Scraping {book}...")
        start = time.time()
        
        verses = scrape_book(book, callback=lambda v: print(f"  {v.reference}"))
        
        # Save to JSON
        with open(output_file, 'w') as f:
            json.dump([v.to_dict() for v in verses], f, indent=2, ensure_ascii=False)
        
        elapsed = time.time() - start
        print(f"✅ {book}: {len(verses)} verses in {elapsed:.1f}s")

# Uncomment to start scraping (this will take HOURS):
scrape_with_persistence()

In [ ]:
# Test the scraper
verse_data = scrape_verse("https://biblehub.com/1_peter/1-1.htm")

print(f"Reference: {verse_data.reference}")
print(f"Book: {verse_data.book}, Chapter: {verse_data.chapter}, Verse: {verse_data.verse}")
print()

print("=" * 60)
print("TRANSLATIONS")
print("=" * 60)
for t in verse_data.translations:
    print(f"\n[{t.version}]")
    print(t.text)

print()
print("=" * 60)
print("GREEK WORDS")
print("=" * 60)
for g in verse_data.greek_words:
    print(f"\n'{g.english_word}' -> {g.word} ({g.transliteration}) - Strong's {g.strongs_number}")
    if g.part_of_speech:
        print(f"  Part of Speech: {g.part_of_speech}")
    if g.definition:
        print(f"  Definition: {g.definition}")

print()
print("=" * 60)
print(f"CROSS REFERENCES ({len(verse_data.cross_references)} total)")
print("=" * 60)
for cr in verse_data.cross_references:  # Show ALL cross-references
    print(f"\n{cr.reference}")
    if cr.text:
        print(f"  {cr.text}")

In [ ]:
# Export as JSON
print(json.dumps(verse_data.to_dict(), indent=2, ensure_ascii=False))

In [73]:
def verse_exists(book: str, chapter: int, verse: int) -> bool:
    """Check if a verse exists on BibleHub (returns False for 404)."""
    url = f"https://biblehub.com/{book}/{chapter}-{verse}.htm"
    try:
        response = requests.head(url, allow_redirects=True)
        return response.status_code == 200
    except requests.RequestException:
        return False


for i in range(100):
  print(verse_exists("genesis", 1, i))

False
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True


KeyboardInterrupt: 

In [75]:
import json

FILE_A = "bible_structure.json"
FILE_B = "bible_structure2.json"

with open(FILE_A, "r", encoding="utf-8") as f:
    a = json.load(f)

with open(FILE_B, "r", encoding="utf-8") as f:
    b = json.load(f)

all_books = sorted(set(a) | set(b))
mismatch = False

for book in all_books:
    if book not in a:
        print(f"[MISSING IN A] {book}")
        mismatch = True
        continue

    if book not in b:
        print(f"[MISSING IN B] {book}")
        mismatch = True
        continue

    chapters_a = a[book]
    chapters_b = b[book]

    if len(chapters_a) != len(chapters_b):
        print(
            f"[CHAPTER COUNT] {book}: "
            f"{len(chapters_a)} != {len(chapters_b)}"
        )
        mismatch = True

    for i, (va, vb) in enumerate(
        zip(chapters_a, chapters_b), start=1
    ):
        if va != vb:
            print(
                f"[VERSE COUNT] {book} {i}: "
                f"{va} != {vb}"
            )
            mismatch = True

if not mismatch:
    print("✔ Structures are identical")
else:
    print("✘ Differences found")


[VERSE COUNT] 1_chronicles 12: 40 != 41
[CHAPTER COUNT] isaiah: 66 != 65
[VERSE COUNT] jeremiah 8: 22 != 23
[VERSE COUNT] jeremiah 9: 26 != 25
✘ Differences found
